In [34]:
!chmod 777 -R data/deriv/fmriprep/

chmod: changing permissions of 'data/deriv/fmriprep/sub-08/ses-se': Operation not permitted
chmod: changing permissions of 'data/deriv/fmriprep/sub-08/ses-se/model': Operation not permitted
chmod: changing permissions of 'data/deriv/fmriprep/sub-08/ses-se/model/sub-08_ses-se_rois.pdf': Operation not permitted
chmod: changing permissions of 'data/deriv/fmriprep/sub-08/ses-se/model/first_level.pkl': Operation not permitted
chmod: changing permissions of 'data/deriv/fmriprep/sub-08/ses-se/model/contrasts.pkl': Operation not permitted
chmod: changing permissions of 'data/deriv/fmriprep/sub-08/ses-se/func': Operation not permitted
chmod: changing permissions of 'data/deriv/fmriprep/sub-08/ses-se/func/sub-08_ses-se_task-stop_run-2_events.tsv': Operation not permitted
chmod: changing permissions of 'data/deriv/fmriprep/sub-08/ses-se/func/sub-08_ses-se_task-stop_run-1_events.tsv': Operation not permitted
chmod: changing permissions of 'data/deriv/fmriprep/sub-08/ses-se/func/sub-08_ses-se_task-

In [27]:
!fmriprep ./data/raw/bids ./data/deriv2/ participant \
--participant-label 07 \
--no-freesurfer \
-w ./workflow_folders \
--n_cpus 8 \
--mem-mb 100000 \
-v \
--output-space T1w template \
--template mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm \
--ants-coreg \
--template-resampling-grid native > $(date +"run_%Y%m%d_%H%M%S.txt")

/usr/local/miniconda/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/local/miniconda/lib/python3.6/site-packages/bids/grabbids/__init__.py:6: FutureWarning: grabbids has been renamed to layout in version 0.6.5, and will be removed in version 0.8
  warnings.warn("grabbids has been renamed to layout in version 0.6.5, and will be removed in version 0.8", FutureWarning)
/usr/local/miniconda/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/local/miniconda/lib/python3.6/site-packages/bids/grabbids/__init__.py:6: FutureWarning: grabbids has been renamed to layout in version 0.6.5, and will be removed in version 0.8
  warnings.warn("grabbids has been renamed to layout in version 0.6.5, and will be removed in version 0.8", FutureW

In [ ]:
## Don't forget to change permissions!

In [13]:
!ls -la data/deriv/fmriprep

total 2800
drwxr-xr-x 21 neuro users   4096 Nov 11 15:14 .
drwxr-xr-x  4 neuro users   4096 Nov  4 12:29 ..
-rw-r--r--  1 neuro users    487 Nov 11 15:14 dataset_description.json
drwxr-xr-x  2 neuro users   4096 Nov  5 08:08 logs
drwxr-xr-x  6 neuro users   4096 Nov 11 15:14 sub-01
-rw-r--r--  1 neuro users 153455 Nov 11 15:14 sub-01.html
drwxr-xr-x  6 neuro users   4096 Nov 11 15:14 sub-02
-rw-r--r--  1 neuro users 153430 Nov 11 15:14 sub-02.html
drwxr-xr-x  6 neuro users   4096 Nov 11 15:14 sub-03
-rw-r--r--  1 neuro users 153455 Nov 11 15:14 sub-03.html
drwxr-xr-x  6 neuro users   4096 Nov 11 15:14 sub-04
-rw-r--r--  1 neuro users 153470 Nov 11 15:14 sub-04.html
drwxr-xr-x  6 neuro users   4096 Nov 11 15:14 sub-05
-rw-r--r--  1 neuro users 153455 Nov 11 15:14 sub-05.html
drwxr-xr-x  6 neuro users   4096 Nov 11 15:14 sub-06
-rw-r--r--  1 neuro users 153360 Nov 11 15:14 sub-06.html
drwxr-xr-x  6 neuro users   4096 Nov 11 15:14 sub-07
-rw-r--r--  1 neuro users 153455 Nov 11 15:14 sub-0

In [14]:
!chmod 777 -R data/deriv/fmriprep

In [4]:
!ls workflow_folders/fmriprep_wf/single_subject_01_wf

about		   func_preproc_ses_me_task_stop_run_1_echo_1_wf
anat_preproc_wf    func_preproc_ses_me_task_stop_run_2_echo_1_wf
bids_info	   func_preproc_ses_me_task_stop_run_3_echo_1_wf
bidssrc		   func_preproc_ses_se_task_stop_run_2_wf
ds_report_about    func_preproc_ses_se_task_stop_run_3_wf
ds_report_summary  summary


In [7]:
!ls -la workflow_folders/fmriprep_wf/single_subject_*_wf

workflow_folders/fmriprep_wf/single_subject_01_wf:
total 36
drwxr-xr-x  9 neuro users 4096 Nov  2 09:15 .
drwxr-xr-x 20 neuro users 4096 Oct 22 19:34 ..
drwxr-xr-x  3 neuro users 4096 Nov  2 08:41 about
drwxr-xr-x 13 neuro users 4096 Oct 22 21:12 anat_preproc_wf
drwxr-xr-x  3 neuro users 4096 Oct 22 18:55 bids_info
drwxr-xr-x  3 neuro users 4096 Oct 22 18:55 bidssrc
drwxr-xr-x  3 neuro users 4096 Nov  2 08:44 ds_report_about
drwxr-xr-x  3 neuro users 4096 Nov  2 08:44 ds_report_summary
drwxr-xr-x  3 neuro users 4096 Oct 22 18:55 summary

workflow_folders/fmriprep_wf/single_subject_02_wf:
total 36
drwxr-xr-x  9 neuro users 4096 Nov  2 09:15 .
drwxr-xr-x 20 neuro users 4096 Oct 22 19:34 ..
drwxr-xr-x  3 neuro users 4096 Nov  1 14:33 about
drwxr-xr-x 13 neuro users 4096 Oct 22 21:59 anat_preproc_wf
drwxr-xr-x  3 neuro users 4096 Oct 22 19:15 bids_info
drwxr-xr-x  3 neuro users 4096 Oct 22 19:15 bidssrc
drwxr-xr-x  3 neuro users 4096 Nov  1 15:42 ds_report_about
drwxr-xr-x  3 neuro users 4